In [1]:
import re
import os
import time
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [14]:


csv_name = "E0.csv"
get_df(csv_name)

,HomeTeam,AwayTeam,FTHG,FTAG
0,Burnley,Man City,0,3
1,Arsenal,Nott'm Forest,2,1
2,Bournemouth,West Ham,1,1
3,Brighton,Luton,4,1
4,Everton,Fulham,0,1
...,...,...,...,...
375,Crystal Palace,Aston Villa,5,0
376,Liverpool,Wolves,2,0
377,Luton,Fulham,2,4
378,Man City,West Ham,3,1


In [15]:
data_df = df[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
data_df

,HomeTeam,AwayTeam,FTHG,FTAG
0,Burnley,Man City,0,3
1,Arsenal,Nott'm Forest,2,1
2,Bournemouth,West Ham,1,1
3,Brighton,Luton,4,1
4,Everton,Fulham,0,1
...,...,...,...,...
375,Crystal Palace,Aston Villa,5,0
376,Liverpool,Wolves,2,0
377,Luton,Fulham,2,4
378,Man City,West Ham,3,1


In [383]:

def get_df(csv_name):
    df = pd.read_csv(csv_name)
    # return df[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
    return df 


def do_work(data_df):
    rez = {"+":0, "-": 0}
    indexes_plus = []
    indexes_minus = []
    for inx in data_df.index:
        try:
        
            target_match = data_df.loc[inx]
            h_team = target_match["HomeTeam"]
            a_team = target_match["AwayTeam"]
            
            last_matches = data_df.iloc[:inx]
           
            some_f_h = last_matches[
                      # (last_matches["HomeTeam"] == h_team )
                      # | 
                        (last_matches["AwayTeam"] == h_team ) 
                    ].tail(1).values[0] 
            h_team_score = some_f_h[2] if some_f_h[0] == h_team else some_f_h[3]
            h_team_prop = some_f_h[3] if some_f_h[0] == h_team else some_f_h[2]
            
            some_f_a = last_matches[
                      (last_matches["HomeTeam"] == a_team )
                      # | 
                      #   (last_matches["AwayTeam"] == a_team ) 
                    ].tail(1).values[0] 
            a_team_score = some_f_a[3] if some_f_a[1] == a_team else some_f_a[2]
            a_team_prop = some_f_a[2] if some_f_a[1] == a_team else some_f_a[3]
    
            
            if some_f_h.shape[0] < 1 and some_f_a.shape[0] < 1:
                continue
    
            # забито по 0 голов
            # if h_team_score == a_team_score :
            #     if h_team_prop - a_team_prop <= -1:
            #         indexes_plus.append(inx)
            #     elif h_team_prop - a_team_prop >= 1:
            #     # else:
            #         indexes_minus.append(inx)
            # пропущено по 0 голов
            if h_team_prop == 2 and a_team_prop == 0 :
                if h_team_score - a_team_score >= -1:
                    indexes_plus.append(inx)
                # elif h_team_score - a_team_score <= -2:
                else:
                    indexes_minus.append(inx)
        except IndexError: 
            continue
    # print(rez)
    # print(rez["+"] / (rez["+"] + rez["-"]))
    return (indexes_plus, indexes_minus)

def predict(full_df, ind_p, ind_m):
    fora = full_df["FTHG"] - full_df["FTAG"] + full_df["AHCh"]
    # (100 / (a + b) ) * a * 1.85
    rez_w_p = fora[ind_p][fora[ind_p] >= 0.5 ].shape[0]
    rez_l_p = fora[ind_p][fora[ind_p] <= -0.5 ].shape[0]
    tot = 100 / (rez_w_p + rez_l_p) * rez_w_p * 1.85 - 100 if (rez_w_p + rez_l_p) > 0 else 0
    print(fora[ind_p][fora[ind_p] >= 0.5 ].shape, fora[ind_p][fora[ind_p] == 0.25 ].shape)
    print(fora[ind_p][fora[ind_p] <= -0.5 ].shape, fora[ind_p][fora[ind_p] == -0.25 ].shape)
    print(fora[ind_p][fora[ind_p] == 0 ].shape)
    print(f"{tot:0.2f}") 

    rez_w_m = fora[ind_m][fora[ind_m] >= 0.5 ].shape[0]
    rez_l_m = fora[ind_m][fora[ind_m] <= -0.5 ].shape[0]
    tot_m = 100 / (rez_w_m + rez_l_m) * rez_l_m * 1.85 - 100 if (rez_w_m + rez_l_m) > 0 else 0
    print("-"*5)
    print(fora[ind_m][fora[ind_m] >= 0.5 ].shape, fora[ind_m][fora[ind_m] == 0.25 ].shape)
    print(fora[ind_m][fora[ind_m] <= -0.5 ].shape, fora[ind_m][fora[ind_m] == -0.25 ].shape)
    print(fora[ind_m][fora[ind_m] == 0 ].shape)
    print(f"{tot_m:0.2f}")  
    


In [384]:
# csv_names = os.listdir("../../csv_data/22_23/")
# csv_names = ("22_23/E0.csv", "23_24/E0.csv")
# csv_names = ("22_23/E1.csv", "23_24/E1.csv")
csv_names = ("22_23/E2.csv", "23_24/E2.csv")
# csv_names = ("22_23/I1.csv", "23_24/I1.csv")
# csv_names = ("22_23/I2.csv", "23_24/I2.csv")

for i in csv_names:
    df = get_df(f"../../csv_data/{i}")
    # df = get_df(f"../../csv_data/23_24/{i}")
    data_df = df[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
    print(i)
    rez_h, rez_a = do_work(data_df)
    print("="*11)
    predict(df, rez_h, rez_a)
    print()

22_23/E2.csv
(16,) (2,)
(8,) (0,)
(3,)
23.33
-----
(4,) (1,)
(5,) (1,)
(0,)
2.78

23_24/E2.csv
(12,) (3,)
(18,) (1,)
(2,)
-26.00
-----
(3,) (1,)
(1,) (2,)
(3,)
-53.75



In [183]:
df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E2,30/07/2022,15:00,Accrington,Charlton,2,2,D,0,1,...,1.92,0.00,2.05,1.75,2.08,1.83,2.16,1.83,2.06,1.77
1,E2,30/07/2022,15:00,Bristol Rvs,Forest Green,1,2,A,0,0,...,1.75,-0.25,2.10,1.77,2.09,1.82,2.13,1.82,2.07,1.76
2,E2,30/07/2022,15:00,Cambridge,Milton Keynes Dons,1,0,H,1,0,...,1.61,0.25,1.70,2.10,1.76,2.14,1.86,2.14,1.78,2.05
3,E2,30/07/2022,15:00,Cheltenham,Peterboro,2,3,A,2,0,...,2.04,0.25,1.93,1.93,1.98,1.90,1.98,1.99,1.89,1.92
4,E2,30/07/2022,15:00,Derby,Oxford,1,0,H,0,0,...,1.81,-0.25,1.75,2.05,1.79,2.10,1.86,2.14,1.79,2.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,E2,07/05/2023,12:00,Lincoln,Shrewsbury,1,0,H,0,0,...,1.81,-0.50,1.95,1.90,1.98,1.87,2.00,1.94,1.95,1.84
548,E2,07/05/2023,12:00,Oxford,Accrington,1,2,A,1,0,...,2.11,-1.25,1.90,1.95,1.94,1.90,1.96,2.02,1.90,1.90
549,E2,07/05/2023,12:00,Port Vale,Plymouth,1,3,A,1,2,...,2.18,0.75,1.95,1.90,1.94,1.91,2.04,1.94,1.96,1.85
550,E2,07/05/2023,12:00,Portsmouth,Wycombe,2,2,D,1,1,...,2.06,-0.25,1.80,2.00,1.90,1.95,1.90,2.06,1.83,1.95


In [184]:
fora = df["FTHG"] - df["FTAG"] + df["AHCh"]
fora

0      0.00
1     -1.25
2      1.25
3     -0.75
4      0.75
       ... 
547    0.50
548   -2.25
549   -1.25
550   -0.25
551    1.00
Length: 552, dtype: float64

In [196]:
a = [35, 43, 44, 67, 148, 178, 214, 219, 257, 300, 305, 310, 327, 344, 356, 381, 397, 409, 417, 429, 440, 442, 450, 476, 535, 548]
print(fora[a][fora[a] >= 0.5 ].shape, fora[a][fora[a] == 0.25 ].shape)
print(fora[a][fora[a] <= -0.5 ].shape, fora[a][fora[a] == -0.25 ].shape)
print(fora[a][fora[a] == 0 ].shape)


(10,) (0,)
(13,) (2,)
(1,)


In [378]:
a = 43
b = 36
c = (100 / (a + b) ) * a * 1.85
c

100.69620253164558

In [123]:
df.loc[[10, 17, 27, 54, 86, 97, 108, 110, 115, 118, 148, 205, 225, 247, 248, 272, 298, 340, 343]]

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
10,E0,13/08/2022,12:30,Aston Villa,Everton,2,1,H,1,0,...,1.74,-0.75,2.05,1.85,2.07,1.87,2.07,1.90,2.02,1.86
17,E0,14/08/2022,14:00,Nott'm Forest,West Ham,1,0,H,1,0,...,1.96,0.50,1.90,2.00,1.93,1.99,1.96,2.04,1.92,1.95
27,E0,21/08/2022,14:00,West Ham,Brighton,0,2,A,0,1,...,1.86,0.00,1.83,2.07,1.86,2.08,1.86,2.12,1.81,2.08
54,E0,03/09/2022,15:00,Nott'm Forest,Bournemouth,2,3,A,2,0,...,1.73,-0.25,1.87,2.03,1.89,2.04,1.94,2.08,1.87,2.00
86,E0,10/10/2022,20:00,Nott'm Forest,Aston Villa,1,1,D,1,1,...,1.94,0.50,1.89,2.01,1.89,2.03,1.90,2.08,1.85,2.03
97,E0,18/10/2022,19:30,Brighton,Nott'm Forest,0,0,D,0,0,...,2.13,-1.25,1.91,1.99,1.93,2.00,1.93,2.03,1.88,1.99
108,E0,22/10/2022,15:00,Man City,Brighton,3,1,H,2,0,...,2.63,-1.75,1.89,2.01,1.95,1.95,1.95,2.04,1.88,1.99
110,E0,23/10/2022,14:00,Aston Villa,Brentford,4,0,H,3,0,...,1.89,-0.50,2.07,1.83,2.08,1.85,2.12,1.88,2.06,1.82
115,E0,24/10/2022,20:00,West Ham,Bournemouth,2,0,H,1,0,...,1.90,-1.00,2.11,1.79,2.12,1.83,2.14,1.85,2.06,1.81
118,E0,29/10/2022,15:00,Brentford,Wolves,1,1,D,0,0,...,1.90,-0.25,1.89,2.01,1.90,2.03,1.92,2.05,1.87,2.00
